In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.ticktype import TickTypeEnum
from ibapi.order import Order
import backtrader as bt
import pytz, tzlocal
from bs4 import BeautifulSoup
import re
from ibapi.ticktype import TickTypeEnum
from datetime import datetime
from time import sleep, strftime, localtime  

import pandas as pd
import numpy as np

In [2]:
class TestApp(EWrapper, EClient):
    
    def __init__(self):
        EClient.__init__(self,self)

    def error(self, reqId, errorCode, errorString):
        print ('Error: ', reqId, errorCode, ' ', errorString)
        
    def fundamentalData(self,reqId, data ):
        global annual
        global quarter
   
        parser = BeautifulSoup(data, 'lxml')

        mapp = dict()
        for item in parser.findAll('mapitem'):
            mapp[item['coaitem']] = item.text
        
    
        #ANNUAL
        index = [];columns = []
        for item in parser.findAll('annualperiods')[0].findAll('fiscalperiod'):
            index.append(datetime.strptime(item['fiscalyear'], '%Y').year)
    
        for i in parser.findAll('annualperiods')[0].findAll('fiscalperiod')[0].findAll('lineitem'):
            columns.append(mapp[i['coacode']])

        annual = pd.DataFrame(index = index, columns = columns)
        annual.index.name = 'Annual'
        for item in parser.findAll('annualperiods')[0].findAll('fiscalperiod'):
            for i in item.findAll('lineitem'):
                try:
                    annual.loc[datetime.strptime(item['fiscalyear'], '%Y').year,mapp[i['coacode']]] = float(i.text)
                except:
                    annual.loc[datetime.strptime(item['fiscalyear'], '%Y').year,mapp[i['coacode']]] = np.nan
        
        # QUARTER
        index_q = [];columns_q = []
        for item in parser.findAll('interimperiods')[0].findAll('fiscalperiod'):
            index_q.append(datetime.strptime(item['enddate'], '%Y-%m-%d').date())
    
        for i in parser.findAll('interimperiods')[0].findAll('fiscalperiod')[0].findAll('lineitem'):
            columns_q.append(mapp[i['coacode']])

        quarter = pd.DataFrame(index = index_q, columns = columns_q)
        quarter.index.name = 'Quarter'
        for item in parser.findAll('interimperiods')[0].findAll('fiscalperiod'):
            for i in item.findAll('lineitem'):
                try:
                    quarter.loc[datetime.strptime(item['enddate'], '%Y-%m-%d').date(),mapp[i['coacode']]] = float(i.text)
                except:
                    quarter.loc[datetime.strptime(item['enddate'], '%Y-%m-%d').date(),mapp[i['coacode']]] = np.nan
    
    
#     def tickPrice(self,reqId, tickType, price, attrib):
#         global df
#         df = pd.DataFrame(data = [reqId, TickTypeEnum.to_str(tickType), price], index = ['Ticker Id','tickType','Price'])
#         #print (df)
#         #print ('Tick Price.','\n','Ticker Id:', reqId,'\n','tickType', TickTypeEnum.to_str(tickType),'\n', 'Price', price,'\n')

#     def tickSize(self,reqId, tickType, size):
        
#         df2 = pd.DataFrame(data = [reqId, TickTypeEnum.to_str(tickType), size], index = ['Ticker Id','tickType','Size'])
#         print ('Tick Size.','\n','Ticker Id:', reqId,'\n','tickType', TickTypeEnum.to_str(tickType),'\n','Size', size,'\n')
        
        
    def nextValidId(self, orderId):
        global orderid
        self.nextValidId = orderId
        orderid = orderId
        self.start()


In [3]:
def create_contract(symbol, sec_type, exch, prim_exch, curr):

    contract = Contract()
    contract.symbol = symbol
    contract.secType = sec_type
    contract.exchange = exch
    contract.primaryExchange = prim_exch
    contract.currency = curr
    
    return contract

def create_order(orderId, order_type, quantity, action):

    order = Order()
    order.orderId
    order.orderType = order_type
    order.totalQuantity = quantity
    order.action = action
    
    return order

In [4]:
contract = create_contract(symbol = 'AAPL',
                           sec_type = 'STK',
                           exch = 'SMART',
                           prim_exch = 'NASDAQ',
                           curr = 'USD')

In [5]:
# df = pd.DataFrame(index = ['Ticker Id','tickType','Price'])
# orderid = None
annual = pd.DataFrame()
quarter = pd.DataFrame()

In [6]:
app = TestApp()
app.connect(host='127.0.0.1',port=7497, clientId=0)


In [7]:
# app.reqMarketDataType(4) # switch to delayed frozen data if live is not available
# app.reqMktData( reqId=0, 
#                 contract=contract, 
#                 genericTickList='', 
#                 snapshot=True, 
#                 regulatorySnapshot =False, 
#                 mktDataOptions=[])
# app.reqIds(0)
app.reqFundamentalData( reqId = 0,
                        contract = contract,
                        reportType = 'ReportsFinStatements',
                        fundamentalDataOptions = []
                        )   


In [ ]:
app.run()

Error:  -1 2104   Market data farm connection is OK:eufarm
Error:  -1 2104   Market data farm connection is OK:cashfarm
Error:  -1 2104   Market data farm connection is OK:usfarm
Error:  -1 2106   HMDS data farm connection is OK:fundfarm
Error:  -1 2106   HMDS data farm connection is OK:ushmds
Error:  -1 2158   Sec-def data farm connection is OK:secdefnj


In [8]:
order = create_order(order_type = 'MKT',
                     quantity = 10, 
                     action = 'BUY')

In [9]:
print ('Creating Order: ')
print (' Ticker: ',contract.m_symbol)
print (' SecType: ',contract.m_secType)
print (' Exchange: ',contract.m_exchange)
print (' primExchange: ',contract.m_primaryExch)
print (' Order Quantity: ',order.m_totalQuantity,'\n','Order Type: ', order.m_orderType,'\n','Order Action: ',order.m_action)

Creating Order: 
 Ticker:  AAPL
 SecType:  STK
 Exchange:  SMART
 primExchange:  NASDAQ
 Order Quantity:  10 
 Order Type:  MKT 
 Order Action:  BUY
